# SUPPORT Hyperparameter Tuning

This notebook runs through hyperparameter tuning for the SUPPORT dataset. For this we use the Optuna library.

For users with limited computational power OR with no access to MIMIC datasets.

We validate our hyperparameter tuning results on our training dataset metrics - This is isn't optimal as usually it would be validated on a separate validation set. Hard to create an appropriate validation set in this instance as we would require the distributions for each variable column to look similar between training & validation.

NOTE: There are known limitations that are explained as they come up in these markdown cells.

In [1]:
# Standard imports
import numpy as np
import pandas as pd
import torch

# VAE is in other folder as well as opacus adapted library
import sys

sys.path.append("../")

# Opacus support for differential privacy
from opacus.utils.uniform_sampler import UniformWithReplacementSampler

# For the SUPPORT dataset
from pycox.datasets import support

# For VAE dataset formatting
from torch.utils.data import TensorDataset, DataLoader

# VAE functions
from VAE import Decoder, Encoder, VAE

# Utility file contains all functions required to run notebook
from utils import (
    support_pre_proc,
    plot_elbo,
    plot_likelihood_breakdown,
    plot_variable_distributions,
    reverse_transformers,
)
from metrics import distribution_metrics

# For hyperparameter tuning as well as saving different trial objects
import optuna
import pickle


## Data Loading & Column Definitions

First we load in the SUPPORT dataset from pycox datasets. Then we define the continuous and categorical columns in that dataset

In [2]:
# Load in the support data
data_supp = support.read_df()

# Save the original columns

original_continuous_columns = ["duration"] + [f"x{i}" for i in range(7, 15)]
original_categorical_columns = ["event"] + [f"x{i}" for i in range(1, 7)]



## Data Pre-Processing

Data can be pre-processed in 2 ways. Either we use <b>"standard"</b> option which performs a standard scaler on continuous variables - This has known limitations as:

- Data in tables is usually non-gaussian and SynthVAE implements a gaussian loss, so this will perform worse unless the data is KNOWN to follow a gaussian distribution already.

Or we use the second option of <b>"GMM"</b>. This performs a variational gaussian mixture model to scale the data & transform it to a gaussian distribution. We use a maximum number of clusters of 10 but the variational method will select the best number of clusters for that continuous variable. This also has known limitations:

- 10 Clusters is arbitrary and may not be enough for certain variables.
- We are fitting a model to transform the data and hence we are approximating before model is trained. This will lose fidelity as the distribution will not be transformed perfectly.

SUPPORT is a limited dataset as it has no missingness (which our model currently does NOT handle) and it has no datetime columns or other data types. Be wary drawing any conclusions from this set due to these constraints as well as the dataset size. Testing/training new models with this set could be useful but conclusive results should be tested on other sets.

In [3]:
#%% -------- Data Pre-Processing -------- #

pre_proc_method = "GMM"

(
    x_train,
    data_supp,
    reordered_dataframe_columns,
    continuous_transformers,
    categorical_transformers,
    num_categories,
    num_continuous,
) = support_pre_proc(data_supp=data_supp, pre_proc_method=pre_proc_method)



C:\Users\dxb085\Anaconda3\envs\SynthVAE\lib\site-packages\pandas\core\frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
C:\Users\dxb085\Anaconda3\envs\SynthVAE\lib\site-packages\sklearn\mixture\_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn('Initialization %d did not converge. '
C:\Users\dxb085\Anaconda3\envs\SynthVAE\lib\site-packages\sklearn\mixture\_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn('Initialization %d did not converge. '
C:\Users\dxb085\Anaconda3\envs\Syn

## Creation & Training of VAE.

We can adapt certain parameters of the model e.g. batch size, latent dimension size etc. This model implements early stopping and these values can be adapted.

We can also activate differential privacy by implementing dp-sgd through the opacus library.

The user defined parameters are defined first and these are arbitrary. For example you could change batch size as well as other variables and if you wanted to do this then you simply move batch size into the objective function in the cell below and then follow the Optuna guidelines on creating a hyperparameter selection.

NOTE: training can be fail and cause errors if the hyperparameter values are not chosen carefully. In this example learning rate was left as <i>1e-3</i> rather than adapted as giving it a selection lead to errors in the training of the encoder - something to watch out for

In [ ]:
#%% -------- Create & Train VAE -------- #

# User defined parameters

# General training
batch_size = 32
n_epochs = 5
logging_freq = 1  # Number of epochs we should log the results to the user
patience = 5  # How many epochs should we allow the model train to see if
# improvement is made
delta = 10  # The difference between elbo values that registers an improvement
filepath = None  # Where to save the best model


# Privacy params
differential_privacy = False  # Do we want to implement differential privacy
sample_rate = 0.1  # Sampling rate
noise_scale = None  # Noise multiplier - influences how much noise to add
target_eps = 1  # Target epsilon for privacy accountant
target_delta = 1e-5  # Target delta for privacy accountant

# Define the metrics you want the model to evaluate

gower = False

# Prepare data for interaction with torch VAE
Y = torch.Tensor(x_train)
dataset = TensorDataset(Y)

generator = None
sample_rate = batch_size / len(dataset)
data_loader = DataLoader(
    dataset,
    batch_sampler=UniformWithReplacementSampler(
        num_samples=len(dataset), sample_rate=sample_rate, generator=generator
    ),
    pin_memory=True,
    generator=generator,
)



# Setting Up Optuna Hyperparameter Tuning Objective Function

See markdown above for details

In [ ]:
# -------- Define our Optuna trial -------- #


def objective(
    trial,
    gower,
    differential_privacy=False,
    target_delta=1e-3,
    target_eps=10.0,
    n_epochs=50,
):

    latent_dim = trial.suggest_int("Latent Dimension", 2, 128, step=2)  # Hyperparam
    hidden_dim = trial.suggest_int("Hidden Dimension", 32, 1024, step=32)  # Hyperparam

    encoder = Encoder(x_train.shape[1], latent_dim, hidden_dim=hidden_dim)
    decoder = Decoder(latent_dim, num_continuous, num_categories=num_categories)

    lr = trial.suggest_float("Learning Rate", 1e-3, 1e-2, step=1e-5)
    vae = VAE(encoder, decoder, lr=1e-3)  # lr hyperparam

    C = trial.suggest_int("C", 10, 1e4, step=50)  # Clipping hyperparam

    if differential_privacy == True:
        (
            training_epochs,
            log_elbo,
            log_reconstruction,
            log_divergence,
            log_categorical,
            log_numerical,
        ) = vae.diff_priv_train(
            data_loader,
            n_epochs=n_epochs,
            C=C,  # Hyperparam
            target_eps=target_eps,
            target_delta=target_delta,
            sample_rate=sample_rate,
        )
        print(f"(epsilon, delta): {vae.get_privacy_spent(target_delta)}")

    else:

        (
            training_epochs,
            log_elbo,
            log_reconstruction,
            log_divergence,
            log_categorical,
            log_numerical,
        ) = vae.train(data_loader, n_epochs=n_epochs)

    # -------- Synthetic Data Generation -------- #

    synthetic_sample = vae.generate(data_supp.shape[0])

    if torch.cuda.is_available():
        synthetic_sample = pd.DataFrame(
            synthetic_sample.cpu().detach(), columns=reordered_dataframe_columns
        )
    else:
        synthetic_sample = pd.DataFrame(
            synthetic_sample.detach(), columns=reordered_dataframe_columns
        )

    # Reverse the transformations

    synthetic_supp = reverse_transformers(
        synthetic_set=synthetic_sample,
        data_supp_columns=data_supp.columns,
        cont_transformers=continuous_transformers,
        cat_transformers=categorical_transformers,
        pre_proc_method=pre_proc_method,
    )
    # -------- SDV Metrics -------- #

    metrics = distribution_metrics(
        gower_bool=gower,
        data_supp=data_supp,
        synthetic_supp=synthetic_supp,
        categorical_columns=original_categorical_columns,
        continuous_columns=original_continuous_columns,
        saving_filepath=None,
        pre_proc_method=pre_proc_method,
    )

    # Optuna wants a list of values in float form

    list_metrics = [metrics[i] for i in metrics.columns]

    print(list_metrics)

    return list_metrics


## Hyperparameter Trials

Here we use optuna to set up a study and run it for a predefined number of trials. If the study has not already been created then set <b>first_run</b> to True. This will then create the study for running.

NOTE: directions show if we are maximising or minimising the metrics we are inputting. Most of SDV metrics require maximizing and that is why <b>directions</b> is set up like this. If you are inputting metrics that require minimizing then you need to set up your directions list accordingly.

In [ ]:
# If there is no study object in your folder then run and save the study so
# It can be resumed if needed

# User parameters

first_run = True  # First run indicates if we are creating a new hyperparam study
saving_filepath = ""  # To save the study if you wish - needs to be .pkl format
n_trials = 3  # Number of trials you want to hyperparameter tune for
loading_filepath = None  # For loading any older study objects - needs to be .pkl format

if first_run == True:

    if gower == True:

        directions = ["maximize" for i in range(8)]

    else:

        directions = ["maximize" for i in range(7)]

    study = optuna.create_study(directions=directions)

else:

    with open("{}".format(loading_filepath), "rb") as f:
        study = pickle.load(f)

study.optimize(
    lambda trial: objective(
        trial,
        gower=gower,
        differential_privacy=differential_privacy,
        target_delta=target_delta,
        target_eps=target_eps,
        n_epochs=n_epochs,
    ),
    n_trials=n_trials,
    gc_after_trial=True,
)  # GC to avoid OOM



## Saving The Study

Here we use pickle to save the study so that it can be loaded up and run from its current point.

If your study is a multi objective study then it will give you multiple <b>best_trials</b> when you use <b>study.best_trials</b>. Depending your weighting for each metric, you can decide which study you will pick as optimal. An example of this is shown in the second cell where each metric is equally important and we average over them using the mean.

In [ ]:
#%% -------- Save The  Study -------- #

with open("{}".format(saving_filepath), "wb") as f:
    pickle.dump(study, f)



In [ ]:
trial_averages = []

for trials in study.best_trials:

    metrics = trials.values
    trial_averages.append(np.mean(metrics))

# Now find the best trial

best_trial = np.argmax(np.asarray(trial_averages))

# Best trial hyperparameters

study.best_trials[best_trial].params
